<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Beer-Cooling-Prediction" data-toc-modified-id="Beer-Cooling-Prediction-1">Beer Cooling Prediction</a></span><ul class="toc-item"><li><span><a href="#The-Goal" data-toc-modified-id="The-Goal-1.1">The Goal</a></span><ul class="toc-item"><li><span><a href="#Issues-to-consider" data-toc-modified-id="Issues-to-consider-1.1.1">Issues to consider</a></span></li></ul></li><li><span><a href="#This-Notebook" data-toc-modified-id="This-Notebook-1.2">This Notebook</a></span><ul class="toc-item"><li><span><a href="#Part-1:-Complete-the-missing-code" data-toc-modified-id="Part-1:-Complete-the-missing-code-1.2.1">Part 1: Complete the missing code</a></span></li><li><span><a href="#Part-2:-Main-Section" data-toc-modified-id="Part-2:-Main-Section-1.2.2">Part 2: Main Section</a></span></li></ul></li><li><span><a href="#Your-Task" data-toc-modified-id="Your-Task-1.3">Your Task</a></span></li></ul></li><li><span><a href="#Part-1:-Complete-the-code" data-toc-modified-id="Part-1:-Complete-the-code-2">Part 1: Complete the code</a></span><ul class="toc-item"><li><span><a href="#1a.-Complete-prepare_cooling_data" data-toc-modified-id="1a.-Complete-prepare_cooling_data-2.1">1a. Complete <code>prepare_cooling_data</code></a></span></li><li><span><a href="#1b.-Clean-data" data-toc-modified-id="1b.-Clean-data-2.2">1b. Clean data</a></span></li><li><span><a href="#1c.-Extract-all-rows-related-to-cooling-phase,-compute-cooling-elapsed-time" data-toc-modified-id="1c.-Extract-all-rows-related-to-cooling-phase,-compute-cooling-elapsed-time-2.3">1c. Extract all rows related to cooling phase, compute cooling elapsed time</a></span></li><li><span><a href="#1d.-Define-temperature-equation" data-toc-modified-id="1d.-Define-temperature-equation-2.4">1d. Define temperature equation</a></span></li></ul></li><li><span><a href="#Non-linear-regression:-computation-and-prediction" data-toc-modified-id="Non-linear-regression:-computation-and-prediction-3">Non-linear regression: computation and prediction</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#NOTE:-there-is-no-code-to-complete-for-those-2-functions" data-toc-modified-id="NOTE:-there-is-no-code-to-complete-for-those-2-functions-3.0.1">NOTE: there is no code to complete for those 2 functions</a></span></li></ul></li></ul></li><li><span><a href="#Part-2:-Main-Section" data-toc-modified-id="Part-2:-Main-Section-4">Part 2: Main Section</a></span><ul class="toc-item"><li><span><a href="#2a.-Set-input-parameters" data-toc-modified-id="2a.-Set-input-parameters-4.1">2a. Set input parameters</a></span></li><li><span><a href="#2b.-Initialize-Hub/OCS-client" data-toc-modified-id="2b.-Initialize-Hub/OCS-client-4.2">2b. Initialize Hub/OCS client</a></span></li><li><span><a href="#2c.-Download-data-from-OCS-with-Data-Views" data-toc-modified-id="2c.-Download-data-from-OCS-with-Data-Views-4.3">2c. Download data from OCS with Data Views</a></span><ul class="toc-item"><li><span><a href="#Find-predefined-data-view-for-Fermenter-#32-and-&quot;Cooling&quot;" data-toc-modified-id="Find-predefined-data-view-for-Fermenter-#32-and-&quot;Cooling&quot;-4.3.1">Find predefined data view for Fermenter #32 and "Cooling"</a></span></li><li><span><a href="#Data-View-Structure" data-toc-modified-id="Data-View-Structure-4.3.2">Data View Structure</a></span></li><li><span><a href="#Get-Interpolated-Data-from-Data-View" data-toc-modified-id="Get-Interpolated-Data-from-Data-View-4.3.3">Get Interpolated Data from Data View</a></span></li></ul></li><li><span><a href="#2d.-Main-Analysis" data-toc-modified-id="2d.-Main-Analysis-4.4">2d. Main Analysis</a></span><ul class="toc-item"><li><span><a href="#Analysis-parameters" data-toc-modified-id="Analysis-parameters-4.4.1">Analysis parameters</a></span></li><li><span><a href="#Plot-data-from-data-view-result-for-brand-of-interest,-keep-only-important-brewing-phase" data-toc-modified-id="Plot-data-from-data-view-result-for-brand-of-interest,-keep-only-important-brewing-phase-4.4.2">Plot data from data view result for brand of interest, keep only important brewing phase</a></span></li><li><span><a href="#Compute-cooling-elapsed-time" data-toc-modified-id="Compute-cooling-elapsed-time-4.4.3">Compute cooling elapsed time</a></span></li><li><span><a href="#Rearrange-dataframe-and-plot-batches-data" data-toc-modified-id="Rearrange-dataframe-and-plot-batches-data-4.4.4">Rearrange dataframe and plot batches data</a></span></li><li><span><a href="#Regression-parameters" data-toc-modified-id="Regression-parameters-4.4.5">Regression parameters</a></span></li></ul></li><li><span><a href="#2e.-Plot-prediction-curve-along-with-actual-data" data-toc-modified-id="2e.-Plot-prediction-curve-along-with-actual-data-4.5">2e. Plot prediction curve along with actual data</a></span></li></ul></li></ul></div>

---
# Beer Cooling Prediction
---

This learning module, Beer Cooling Prediction, focuses on the cooling phase wherein the temperature of the solution drops from 70°F to 30°F.  The data was compiled based on the temperature measurements at the top, the middle, and the bottom of the fermenter:

![Beer Cooling](https://academichub.blob.core.windows.net/images/NB2_Problem_Statement.png)

Attaining a consistent, repeatable cooling profile is critical to the brewing process because the cooling behavior affects the flavor profile of the beers; furthermore, non-ideal cooling behavior can lead to significant process delays. In fact, Deschutes Brewery previously encountered non-ideal cooling behaviors where the temperature spiked when the yeast was harvested. By visualizing the data, Deschutes Brewery was able to identify the source of the issue, correct it, and save 60% of operation time compared to the worst non-ideal cases.  If interested, a more detailed story about the root cause and their corrective action is found in the presentation: [What’s Really Going on with your Beer's Fermentation?](https://www.osisoft.com/Presentations/What-s-Really-Going-on-with-your-Beer-s-Fermentation-/)

In this notebook, we will build a predictive model for the cooling temperature profile to predict when the cooling stage should end for some given brand of beer. 

First, make sure the latest library version for OCS is installed:

In [2]:
try:
    import ocs_academic_hub
except ImportError:
    !pip installocs-academic-hub
    import ocs_academic_hub

Then make sure to import all the required libraries:

In [3]:
# For interaction with OCS
from ocs_academic_hub import HubClient, timer

import configparser
import datetime as dt
from dateutil import parser
from enum import Enum

try:
    import plotly.express as px 
except ImportError:
    !pip install plotly
    import plotly.express as px 
import numpy
import pandas as pd
from scipy.optimize import curve_fit

pd.set_option("display.expand_frame_repr", False)
pd.options.mode.chained_assignment = None

---
## The Goal

Using the same dataset as in the ADF Prediction notebook, we want to identify the cooling stages and compute the elapsed times to align the data for non-linear regression and comparison.

All possible beer brands are for Fermenter Vessel #32 in the time period specified below: 

* Mirror Pond
* Hop Slice
* Pacific Wonderland
* Black Butte           
* Inversion         

We'll start with the following input parameters: 

* Brand: Mirror Pond 
* Temperature sensors: Bottom, Middle and Top
* Training data: 182 days starting at 2017-01-10T00:00 - pick first 2 batches 
* Interval: 10 minutes (00:10:00)

### Issues to consider

* Real process data is dirty (sensors can fail)
* The stages may sometimes have been mislabelled (some examples are "Cooling" sometimes mislabelled as "Diacetyl Rest" or "Maturation" sometimes mislabelled as "Cooling")
* There are different cooling start temperatures for each batch and even for each zone
* There can be various volumes of beer in the fermentors
* Data can have abrupt, aphysical jumps in temperature

---
## This Notebook


### Part 1: Complete the missing code 
Complete TODO sections in the following cells to compute the cooling profile for a vessel zone. The defined functions perform the following tasks
    
* Clean the data by keeping relevant ([1a](#1a.-Complete-prepare_cooling_data)) and removing bad ([1b](1b.-Clean-data)) entries
* Identify the cooling stages ([1c](#1c.-Extract-all-rows-related-to-cooling-phase,-compute-cooling-elapsed-time)) and compute the elapsed times since the start of cooling 
* Fit the cooling data to a heat transfer equation ([1d](#1d.-Define-temperature-equation)) 

### Part 2: Main Section
This section actually calls the functions created in Part 1 and plots the prediction curves against each batch for a given brand of beer.

* Define parameters of interest, like the time granularity for data interpolation, plus the time period over which to do the analysis and the fermenter identifier to analyze data from ([2a](#2a.-Set-input-parameters))
* Use OSIsoft Cloud Services (OCS) to obtain process data from Deschutes ([2b](#2b.-Initialize-Hub/OCS-client) and [2c](#2c.-Download-data-from-OCS-with-Data-Views))
* Set brand of interest, which sensor zones to use and batches for regression. Call functions created in Part 1 in sequence with intermediate plots to follow progression.  ([2d](#2d.-Main-Analysis))
* Plot the cooling prediction curves ([2e](#2e.-Plot-prediction-curve-along-with-actual-data))

**Section 2d and 2e can be repeated iteratively with different kind of beer, temperature sensor zones and batches to check the impact on the prediction curves.**

---
## Your Task 
A few functions in Part 1 contain `TODO` sections in comments. Complete each of them to get a working notebook. If your code is correct, you should see the following graph appear at the bottom of this notebook (at the end of Part 2):

![Beer Cooling Prediction](https://academichub.blob.core.windows.net/images/nb2_solution_graph_v4.png)

---
# Part 1: Complete the code
---
<a id=’section_1’></a>

We want to complete the missing code sections which implement the following specifications:

**Input parameters:**

* Brand of beer
* Which set of temperature sensor to use: bottom, middle, top
* Training data: how many days (starting at 2016-02-07) to consider and set of batch identifier(s) for cooling curve regression

**Output:** 

* Plot with all individual batch cooling profiles against their associated prediction curve 

------------------------------------------------------------------------------------

## 1a. Complete `prepare_cooling_data`
<a id=’section_1a’></a>
This is the function that will be executed to:

* [1] keep only data for brand of interest
* [2] remove rows with missing data, also keep only important phases
* [3] indentify and keep only cooling phase rows, compute cooling time in days for each row (allows to compare batches and necessary for regression)

In [4]:
# Phases important for cooling
IMPORTANT_BREWING_PHASES = ["Fermentation", "Free Rise", "Diacetyl Rest", "Cooling"]

# Sensor positions - for indexing in loops
class Position(Enum):
    Bottom = 1
    Middle = 2
    Top = 3

    def __repr__(self):
        return str(self).replace("Position.", "")


def prepare_cooling_data(all_brands_df, brand):
    """
    Input parameters:
    * brand to consider
    * temperature sensor position to use for computation
    * number of days to compute prediction parameters
    """

    # TODO [1]: write filter expression for all_brands_df, return only rows for `brand` of interest
    # =========== STUDENT BEGIN ==========
    brand_df = all_brands_df[@@@ Your code here @@@]
    # =========== STUDENT END ==========

    # TODO [2]: complete code block within function brand_df_cleanup
    brand_status_df = brand_df_cleanup(brand_df)

    # TODO [3]: complete code of function cooling_data_extraction
    cool_df = cooling_data_extraction(brand_status_df)

    # Verify that it was possible to extract the data for a complete cooling phase
    if len(cool_df) == 0:
        raise Exception(f"Error! no cooling data for brand: {brand}")
    print(f"len(cool_df)={len(cool_df)}")

    return cool_df

## 1b. Clean data 

The function `brand_df_cleanup` removes all rows with invalid (missing data) entries for any of the attributes of interest. To predict the cooling profile for a given brand, one must need the following attributes: Brand, Status, Temperature (Top, Middle, and Bottom zones), and Volume.

Complete the `TODO` section in the function `brand_df_cleanup`.

In [5]:
@timer
def brand_df_cleanup(brand_df):

    # TODO [2]: Remove all rows with missing data (which indicates a sensor collection problem)
    # TIP: Check Notebook #1 ADF Prediction
    # =========== STUDENT BEGIN ==========
    brand_df = brand_df.@@@ Your code here @@@
    # =========== STUDENT END ==========

    # Keep only fermentation or post-fermentation stages
    brand_status_df = brand_df[brand_df["Status"].isin(IMPORTANT_BREWING_PHASES)]

    return brand_status_df

## 1c. Extract all rows related to cooling phase, compute cooling elapsed time

The function for extracting all frames in the cooling phase is `cooling_data_extraction`. 

Overall, `cooling_data_extraction` will extract the temperature information for the cooling stage in a batch in a vessel zone in a fermentor and compute a new column COOLING_TIME with the elapsed time in days since the beginning of cooling. 

In [6]:
@timer
def cooling_data_extraction(brand_status_df):
    #
    # condition for it to be in cooling phase
    # TODO [3]: the condition is that 'Top TIC OUT', 'Middle TIC OUT' and 'Bottom TIC OUT' are above 99.99
    # =========== STUDENT BEGIN ==========
    cool_stage = brand_status_df[
        (brand_status_df["Top TIC OUT"] > 99.99)
        & (@@@ Your code here  @@@)
        & (@@@ Your code here @@@)
    ]
    # =========== STUDENT END ==========

    # TODO [4]: extract first row of each fermentation batch
    # HINT: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
    # =========== STUDENT BEGIN ==========
     cool_start_time = brand_status_df.@@@ Your code here @@@.first()
    # =========== STUDENT END ==========

    # Dataframe to hold cooling data with new COOLING_TIME column holding cooling time in days
    cool_df = pd.DataFrame()
    #
    # For each fermentation batch, compute COOLING_TIME
    for ferm_id in list(start_time_df.index):
        start_time = start_time_df.loc[ferm_id].Timestamp
        print(f"  ==> ferm_id={ferm_id} start={start_time}")
        #
        # Dataframe with rows associated to the current fermentation batch ID
        batch_df = cool_stage_df[cool_stage_df["Fermentation ID"] == ferm_id]
        #
        # Cooling time in unit of days
        batch_df.loc[:, COOLING_TIME] = batch_df["Timestamp"].apply(
            lambda t: (t - start_time) / numpy.timedelta64(1, "D")
        )
        #
        # Collect batch_df into result dataframe
        cool_df = cool_df.append(batch_df)

    return cool_df

**Auxiliary function for plotting** 

Takes a dataframe with {Top|Middle|Bottom} TIC PV columns and returns a dataframe with a single "Temperature" column instead and a new "Position" column with one of the 3 string {Top|Middle|Bottom} to indicate which sensor the temperature value is associated with. 

In [7]:
def arrange_by_position(cool_df):
    position_dfs = []
    for pos in ["Top", "Middle", "Bottom"]:
        position_df = cool_df[
            ["Brand", "Timestamp", "Fermentation ID", f"{pos} TIC PV", "Volume"]
        ].rename(columns={f"{pos} TIC PV": "Temperature", "Fermentation ID": "Ferm_ID"})
        position_df.loc[:, "Position"] = pos
        if COOLING_TIME in cool_df.columns:
            position_df.loc[:, COOLING_TIME] = cool_df[COOLING_TIME]
        position_dfs += [position_df]

    cooling_df = pd.concat(position_dfs)
    return cooling_df

## 1d. Define temperature equation

![Cooling equation](https://academicpi.blob.core.windows.net/images/NB2_Heat_transfer_eqn.png)

The cell below implements this equation for predicting the cooling curve. 

The [scipy.curve_fit (click for reference)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) function is used find the values for the parameters $\alpha$ and $\beta$ (`alpha` and `beta`) using non-linear least square fit. 

**IMPORTANT: To implement the function for $T_{t+1}$ (variable `T_t1`) in the exercise below, you'll need (click link for reference documentation):**

* **[numpy.multiply](https://docs.scipy.org/doc/numpy/reference/generated/numpy.multiply.html?highlight=multiply#numpy.multiply)**
* **[numpy.reciprocal](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reciprocal.html?highlight=reciprocal#numpy.reciprocal)**

In [8]:
def temperature_profile(x, alpha, beta):
    # Unpack x values
    T_t = x[0]  # this is a numpy vector
    Vol = x[1]  # another numpy vector

    # TODO [5]: Implement the function defined in red above
    # TIP: check "IMPORTANT" note above, your code should contain variables alpha, beta, T_t and Vol
    # =========== STUDENT BEGIN ==========
    T_t_1 = @@@ Your code here @@@
    # =========== STUDENT END ==========
    return T_t1

<h1>Non-linear regression: computation and prediction</h1>

Function `compute_cooling_parameters` computes parameter $\alpha$ (`alpha`) and $\beta$ (`beta`) for a given set of fermentation (batch) IDs and temperature zones. 

Function `generate_prediction_df` input are the above regression parameters and a batch dataframe. It returns a new dataframe with the batch data received and additional prediction rows. The "Position" column value for the new rows is "Prediction". 

<h3>NOTE: there is no code to complete for those 2 functions</h3>

In [9]:
############### CURVE FIT REGRESSION BEGIN - DO NOT CHANGE #############


def compute_cooling_parameters(cool_df, brand, temp_sensors, ferm_ids):
    """
    Input parameters:
    * brand to consider
    * temperature sensor position to use for computation
    * number of days to compute prediction parameters
    """

    # vessel zone(s) where cooling profile will be predicted
    sensors_to_use = [repr(pos) for pos in Position if temp_sensors[repr(pos)]]
    # at least one should be true
    if len(sensors_to_use) == 0:
        raise Exception(
            "@@@ Error: at least one of Bottom|Middle|Top should be true in TEMP_SENSORS"
        )
    print(f"sensors to use={sensors_to_use} with batch(es) {ferm_ids}")

    # Only the fermentation IDs and specified sensor zones are kept for regression
    regression_cool_df = cool_df[
        cool_df["Ferm_ID"].isin(ferm_ids) & cool_df["Position"].isin(sensors_to_use)
    ]

    # sort the temperatures in a descending fashion
    regression_cool_df = regression_cool_df.sort_values(
        by=["Temperature"], ascending=False
    )

    # get the y value for the x, this will be used in curve fitting
    regression_cool_df["Temperature_y"] = regression_cool_df["Temperature"].shift(-1)

    # drop the last row
    regression_cool_df = regression_cool_df[:-1]

    # training temperature feature
    x1_train = regression_cool_df["Temperature"].values

    # training Volume feature
    x2_train = regression_cool_df["Volume"].values

    # [temperature, volume]
    x = [x1_train, x2_train]

    # Training of non-linear least squares model
    # Nonlinear curve-fitting pass a tuple in curve fitting
    popt, pcov = curve_fit(
        temperature_profile, x, regression_cool_df["Temperature_y"].values
    )

    # get the coefficients alpha and beta in the model
    alpha = popt[0]
    beta = popt[1]

    # compute mean of maximum elapsed time for all (batch ID, position) pair
    regression_cooling_elapsed = (
        regression_cool_df.groupby(by=["Ferm_ID", "Position"]).max()
    )[COOLING_TIME].mean()
    return (alpha, beta, regression_cooling_elapsed)


############### CURVE FIT REGRESSION END - DO NOT CHANGE #############

In [10]:
def generate_prediction_df(
    alpha,
    beta,
    brand,
    pos_cool_df,
    temp_sensors,
    ferm_id,
    max_cooling_days,
    reg_batch_count=1,
):
    # vessel zone(s) where cooling profile will be predicted
    sensors_to_use = [repr(pos) for pos in Position if temp_sensors[repr(pos)]]

    cool_fermid_df = pos_cool_df[pos_cool_df["Ferm_ID"] == ferm_id]

    # Get the initial point of all temperature curves
    T_0s = (
        cool_fermid_df[cool_fermid_df["Position"].isin(sensors_to_use)]
        .groupby(by=["Position"])
        .first()
    )
    T_0 = T_0s["Temperature"].mean()

    # Volumes - NOTE: all 3 sensors position share the same Volume, so we pick the Top one
    Vol = numpy.repeat(
        cool_fermid_df[cool_fermid_df["Position"].isin(sensors_to_use)][
            "Volume"
        ].values,
        reg_batch_count,
    )

    # Compute the prediction with the start temperature T_0
    T_prediction = [T_0]
    T_t = T_0
    for i in range(1, len(Vol)):
        x = [numpy.atleast_1d(T_t), numpy.atleast_1d(Vol[i])]
        # result is T_t1 computed from T_t
        T_t = temperature_profile(x, alpha, beta)[0]
        T_prediction.append(T_t)

    prediction_df = pd.DataFrame(T_prediction, columns=["Temperature"])
    prediction_df.loc[:, "Brand"] = brand
    prediction_df.loc[:, "Ferm_ID"] = ferm_id
    prediction_df.loc[:, "Position"] = "Prediction"
    prediction_df.loc[:, COOLING_TIME] = numpy.linspace(0.0, max_cooling_days, len(Vol))

    cool_fermid_df = cool_fermid_df.append(prediction_df, sort=True)

    return (T_0, prediction_df, cool_fermid_df)

---
# Part 2: Main Section
---
<a id=’section_2’></a>
Once all functions above are fully implemented and OCS has been initialized, below are the cells to:

[2a](#section_2a) - Set the input parameters   
[2b](#section_2b) - Initialize OCS client<br>
[2c](#section_2c) - Download data from OCS with Data View<br>
[2d](#section_2d) - Call functions of Part 1 to build result dataframe. This is the main analysis.   
[2e](#section_2e) - Plot result data

## 2a. Set input parameters

Specify what fermentor, time period and interpolation interval we want for the data view. 

In [11]:
# time period over which to analyze data
TRAINING_DAYS = 182

# we want to pull this data starting from timestamp "start_index"
START_INDEX = "2017-01-20T15:30"
END_INDEX = (parser.parse(START_INDEX) + dt.timedelta(days=TRAINING_DAYS)).isoformat()

# time granularity
INTERVAL = "00:10:00"

# specify asset 
ASSET_ID = "FV32"

# Additional column name for calculated cooling time in indicated unit
COOLING_TIME = "Cooling (days)"

## 2b. Initialize Hub/OCS client

---
**Executing the next starts the authentication process for your Google account, acting as the Identity Provider for OCS. At the end of a successful login or if you're already logged in with Google, you'll see the following message on a new browser tab:**

**You can now return to the application.**

Simply close this tab and return to your notebook tab. 

---

After a successful login, communication with OCS is done through a HubClient object. Variable `namespace_id` is initialized with the OCS namespace hosting the Deschutes dataset. 

In [12]:
hub_client = HubClient()

dataset = hub_client.current_dataset()
namespace_id = hub_client.namespace_of(dataset)
print(f"Current dataset is {dataset} in namespace_id: {namespace_id}")

## 2c. Download data from OCS with Data Views

**Development tip (WARNING: getting data view results can take up to 30 secs)** 

Development of a notebook involves running code over and over, so you'll want to avoid long running steps when possible. This is why you can run the cell below once, with the resulting dataframe saved in variable `all_brands_df`. If you don't change any of its input parameters, `all_brands_df` is still valid and can be reused when doing your analysis with different parameters. 

Also include a statistical description of `all_brands_df`.

### Find predefined data view for Fermenter #32 and "Cooling"

In [13]:
# Filtered to get the one related to "cooling"
# A list with a single element will be returned, so this element is extracted with `[0]`
dataview_id = hub_client.asset_dataviews(filter="cooling", asset=ASSET_ID)[0]
dataview_id

### Data View Structure 

In [14]:
hub_client.dataview_definition(namespace_id, dataview_id)

### Get Interpolated Data from Data View

In [15]:
# pull data from OCS

# TODO: request a Panda dataframe from OCS
# =========== STUDENT BEGIN ==========
all_brands_df = @@@ Your code here @@@
# =========== STUDENT END ==========

# statistical description of `all_brands_df`, -n/a- stands for "not applicable"
all_brands_df.describe(include="all").fillna('-n/a-')

In [16]:
# preview of dataframe"
all_brands_df.head()

In [17]:
# list brand of beer for cooling with how many rows they appear in
all_brands_df[all_brands_df["Status"] == "Cooling"]["Brand"].value_counts()

In [18]:
# just like in NB1_SOLUTION_ADF_Prediction notebook, computer a Fermentation ID column from `Fermentation_Start_Time`
all_brands_df = all_brands_df.astype({"Fermentation Start Time": "datetime64"})


def fermentation_id(fv_id, t0):
    tt = str(t0).split(" ")
    return f"{fv_id}_{tt[0].replace('-','')}"


all_brands_df["Fermentation ID"] = all_brands_df.apply(
    lambda row: fermentation_id(row["Asset_Id"], row["Fermentation Start Time"]), axis=1
)

## 2d. Main Analysis 

### Analysis parameters

Set brand of beer for the analysis 

In [19]:
# selected brand
BRAND = "Realtime Hops"

### Plot data from data view result for brand of interest, keep only important brewing phase 

In [20]:
tcbdf = arrange_by_position(
    brand_df_cleanup(all_brands_df[all_brands_df["Brand"] == BRAND])
)
fig = px.scatter(
    tcbdf,
    x="Timestamp",
    y="Temperature",
    color="Position",
    hover_name="Ferm_ID",
    title=f"Temperature for {BRAND} brand and phases in {IMPORTANT_BREWING_PHASES}",
)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=7, label="1w", step="day", stepmode="backward"),
                    dict(count=14, label="2w", step="day", stepmode="backward"),
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)
fig.update_traces(hovertemplate=None)
fig.layout.title.font=dict(size=14)
fig.layout.template.layout.hovermode = "x unified"
fig.show()

### Compute cooling elapsed time

As seen in Part 1, `prepare_cooling_data` calls `brand_df_cleanup` and `cooling_data_extraction` to compute a new cooling elapsed time column. 

In [21]:
cool_df = prepare_cooling_data(all_brands_df, BRAND)
cool_df

### Rearrange dataframe and plot batches data

In [22]:
pos_cool_df = arrange_by_position(cool_df)
fig = px.scatter(
    pos_cool_df,
    x=COOLING_TIME,
    y="Temperature",
    color="Position",
    facet_col="Ferm_ID",
    title=f"Cooling Temperature Profile per Batch for {BRAND} Brand",
)
for i in fig["layout"]["annotations"]:
    i["font"] = dict(size=11, color="#000000")
    i["text"] = i["text"].replace("=", "=<br>")
fig.layout.template.layout.hovermode = "x unified"
fig.show()

### Regression parameters

The computation of the non-linear regression curve will pick the set data defined by two criteria: 

* A set of temperature zones with truth value to identify if the zone should contribute (`TEMP_SENSORS`)
* A set of index of batches to consider as regression data (`reg_batches`) 

All data for the regression are from curves matching the two criteria above. 

In [23]:
TEMP_SENSORS = {"Bottom": True, "Middle": True, "Top": True}  # pick all zones

# first and second batches, namely FV32_20170118 and FV32_20170331
reg_batches = [0, 1]

# list of all possible batch ID
batch_ids = list(pos_cool_df["Ferm_ID"].unique())

# list of sensor zones (position) which are selected (True)
sensors_to_use = [repr(pos) for pos in Position if TEMP_SENSORS[repr(pos)]]

# from a list of indices, return associate list of batch ID
def batch_select(ids):
    return [batch_ids[i] for i, _ in enumerate(batch_ids) if i in ids]


print(f"all batch IDs={batch_ids}\nselected batch(es)={batch_select(reg_batches)}")

In [24]:
fig = px.scatter(
    pos_cool_df[
        (
            pos_cool_df["Ferm_ID"].isin(batch_select(reg_batches))
            & (pos_cool_df["Position"].isin(sensors_to_use))
        )
    ],
    x=COOLING_TIME,
    y="Temperature",
    color="Position",
    facet_col="Ferm_ID",
    title=f"Cooling Temperature Profiles for {BRAND} as Regression Input Data",
)
for i in fig["layout"]["annotations"]:
    i["font"] = dict(size=11, color="#000000")
    i["text"] = i["text"].replace("=", "=<br>")
fig.layout.template.layout.hovermode = "x unified"
fig.show()

In [25]:
alpha, beta, max_cooling_days = compute_cooling_parameters(
    pos_cool_df, BRAND, TEMP_SENSORS, batch_select(reg_batches)
)
alpha, beta, max_cooling_days

In [26]:
# build dataframe with all batches and their associated prediction
all_df = pd.DataFrame()
for batch_id in batch_ids:
    _, _, cfdf = generate_prediction_df(
        alpha,
        beta,
        BRAND,
        pos_cool_df,
        TEMP_SENSORS,
        batch_id,
        max_cooling_days,
        len(batch_select(reg_batches)),
    )
    all_df = all_df.append(cfdf)

## 2e. Plot prediction curve along with actual data 

**Note:** You can zoom into the graph to see how the prediction and data actually differ.

In [27]:
fig = px.scatter(
    all_df,
    x=COOLING_TIME,
    y="Temperature",
    color="Position",
    facet_col="Ferm_ID",
    title=f"Cooling Temp with Regression from {batch_select(reg_batches)}"
    f" and {[repr(pos) for pos in Position if TEMP_SENSORS[repr(pos)]]}<br>"
    f"for brand {BRAND}"
)
fig.layout.template.layout.hovermode = "x unified"
fig.layout.title.font = dict(size=14)
for i in fig["layout"]["annotations"]:
    i["font"] = dict(size=11, color="#000000")
    i["text"] = i["text"].replace("=", "=<br>")
fig.show()

**------------------ Your graph will appear above this line if no error occured ------------------**

**------------------------------------- Reference graph below -------------------------------------**
![Beer Cooling Prediction](https://academichub.blob.core.windows.net/images/nb2_solution_graph_v4.png)

**QUESTIONS:**    

1. If the cooling data had outliers, how would you remove them?
2. How would having an erratic cooling profile affect the prediction curve? 
3. Can you use the cooling prediction curve to determine whether a given batch is out-of-spec? How would you do this?
4. Do you think you can use the cooling rate to predict the beer brand?
